##**The Language Model Protein Folder**
<img src="https://github.com/YaadLuria/NanoFoldLM/blob/main/images/logo2.jpeg" height="200" align="right" style="height:240px">

Noa Klugman, Yaad Luria, Elad Applbaum, Eyal Zur

**About The Project**



.

*we recommend to use GPU in order to get lower run times*


#Installs and Imports

In [9]:
%%time

!pip install import-ipynb
import import_ipynb
import os, time


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
CPU times: user 23.2 ms, sys: 7.28 ms, total: 30.5 ms
Wall time: 3.62 s


In [10]:
from google.colab import files
import re
import os
import tensorflow as tf
from tensorflow import keras

In [11]:
from google.colab import files
import re
import os
from IPython.display import clear_output

In [12]:
#@title Clone NanoFoldLM trained model


%%bash
# rm -rf NanoFoldLM
# git clone https://github.com/YaadLuria/NanoFoldLM --quiet

if [ ! -f NanoFoldLMReady ]; then
  # install dependencies
  pip -q install biopython
  pip install py3Dmol
  pip install pdb-tools


  # download model
  if [ ! -d "NanoNet/" ]; then
    git clone https://github.com/YaadLuria/NanoFoldLM --quiet
  fi
  touch NanoFoldLMReady
fi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
# %% [code]
#@title Test git use
os.chdir("/content/NanoFoldLM/NanoFoldLM")
import utils
# if "NanoFoldLM" not in dir():
#   from timeit import default_timer as timer
#   import sys
#   sys.path.insert(0, '/content/NanoFoldLM/')
#   from NanoFoldLM import *

print(utils.NB_MAX_LENGTH) # just for git clone check

140


#Set Parameters

In [ ]:
jobname = "6xw6640" #@param {type:"string"}
jobname = re.sub(r'\W+', '', jobname)[:50]


download_path = "/drive/MyDrive/Hackathon3D" #@param {type:"string"}

sequence = "QVQLQESGGGLVQAGDSLRVSCAASGRTISSSPMGWFRQAPGKEREFVAAISGNGGNTYYLDSVKGRFTTSRDNAKNTVYLQLNNLKPEDTAIYYCAARSRFSAMHLAYRRLVDYDDWGQGTQVTVSSHHHHHH" #@param {type:"string"}
sequence = re.sub("[^A-Z:]", "", sequence.replace("/",":").upper())
sequence = re.sub(":+",":",sequence)
sequence = re.sub("^[:]+","",sequence)
sequence = re.sub("[:]+$","",sequence)



amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
for i,aa in enumerate(sequence):
  if aa.upper() not in amino_acids:
    print(f"Invalid amino acid: {aa} in position {i}")

# if utils.check_valid_protein_sequence(sequence):
#   print(f"Invalid amino acid: {aa}")

seqs = sequence.split(":")
lengths = [len(s) for s in seqs]
length = sum(lengths)



download_pdb = True #@param {type:"boolean"}
print_atom_coordinates = False #@param {type:"boolean"}
embeding_size = "1280" #@param ["320", "480", "640",  "1280", "One_hot_encoding"] {type:"string"}




#Run model

In [14]:
%%time
if embeding_size ==  "One_hot_encoding":
  seq_embeddings = utils.generate_input_one_hot(sequence)
else:
  seq_embeddings = utils.get_esm_embedding_for_protein_sequence(sequence, int(embeding_size))

model_path = f"./Models"
model_dict = {"320": "model_320", "480": "model_480", "640": "model_640",  "1280": "model_1280", "One_hot_encoding": "model_22"}
model = tf.keras.models.load_model(f'{model_path}/{model_dict[embeding_size]}.tf')
import numpy as np
print(seq_embeddings.shape)
predict = model.predict(seq_embeddings)
utils.matrix_to_pdb(sequence, predict[0], jobname)



Using cache found in /root/.cache/torch/hub/facebookresearch_esm_main
array([[[ 0.01330054, -0.04468784,  0.04235788, ..., -0.01141959,
          0.04146361, -0.00996913],
        [ 0.02202126,  0.02767433, -0.05011015, ...,  0.01500191,
          0.02163232,  0.02805064],
        [ 0.03990979, -0.03285698,  0.03470996, ..., -0.02309041,
         -0.02219302,  0.01912651],
        ...,
        [-0.0435444 ,  0.03873615,  0.02576964, ...,  0.00480974,
          0.00695061,  0.02092967],
        [-0.05259109, -0.05094796, -0.05410686, ..., -0.04554587,
         -0.04130068, -0.0030692 ],
        [-0.04560649,  0.05386269, -0.04780322, ...,  0.04434039,
         -0.04806559, -0.00385435]],

       [[-0.01787328, -0.00027128, -0.01600194, ..., -0.03506228,
          0.01289144, -0.02123043],
        [ 0.02800187, -0.05585578, -0.03471456, ...,  0.04906277,
         -0.03539336,  0.04480802],
        [ 0.05231981, -0.04516279,  0.04720613, ..., -0.02195537,
          0.02754182,  0.0287629 

ValueError: ignored

In [ ]:

import py3Dmol
#@title display (optional) {run: "auto"}
pdb_predict = f"{jobname}.pdb"
# pdb_file_path =  "/content/drive/MyDrive/Hackathon3D/Datasets/Train_set/12E8_1.pdb"

with open(pdb_predict, 'r') as pdb_predicts:
  pdb_str = pdb_predicts.read()


from string import ascii_uppercase, ascii_lowercase
alphabet_list = list(ascii_uppercase+ascii_lowercase)
pymol_color_list = ["#33ff33","#00ffff","#ff33cc","#ffff00","#ff9999","#e5e5e5","#7f7fff","#ff7f00",
                    "#7fff7f","#199999","#ff007f","#ffdd5e","#8c3f99","#b2b2b2","#007fff","#c4b200",
                    "#8cb266","#00bfbf","#b27f7f","#fcd1a5","#ff7f7f","#ffbfdd","#7fffff","#ffff7f",
                    "#00ff7f","#337fcc","#d8337f","#bfff3f","#ff7fff","#d8d8ff","#3fffbf","#b78c4c",
                    "#339933","#66b2b2","#ba8c84","#84bf00","#b24c66","#7f7f7f","#3f3fa5","#a5512b"]





def show_pdb(pdb_str, show_sidechains=False, show_mainchains=False,
             color="pLDDT", chains=None, vmin=50, vmax=90,
             size=(800,480), hbondCutoff=4.0,
             Ls=None,
             animate=False):

  if chains is None:
    chains = 1 if Ls is None else len(Ls)
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js', width=size[0], height=size[1])

  if animate:
    view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  else:
    view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  if color == "pLDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':vmin,'max':vmax}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    for n,chain,color in zip(range(chains),alphabet_list,pymol_color_list):
       view.setStyle({'chain':chain},{'cartoon': {'color':color}})
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                  {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  view.zoomTo()
  if animate: view.animate()
  return view

color = "chain" #@param ["confidence", "rainbow", "chain"]
if color == "confidence": color = "pLDDT"
show_sidechains = True #@param {type:"boolean"}
show_mainchains = True #@param {type:"boolean"}
show_pdb(pdb_str, color=color,
         show_sidechains=show_sidechains,
         show_mainchains=show_mainchains,
         Ls=lengths).show()


Compare to solved pdb file

In [ ]:
have_solved_structure = False #@param {type:"boolean"}
pdb_id = "5UZM" #@param {type:"string"}
chain_id = "A" #@param {type:"string"}
if have_solved_structure:
  seq, atoms = utils.get_seq_aa_by_id(pdb_id, chain_id)


  print("********pdb_test********")
  with open(atoms, 'r') as pdb_:
    pdb_test = pdb_.read()
  # print(pdb_test)

  show_pdb(pdb_test, color=color,
        show_sidechains=show_sidechains,
        show_mainchains=show_mainchains,
        Ls=lengths).show()


In [15]:
#@title Download predicted pdb (optional)
if download_pdb:
  from google.colab import files
  files.download(f"{jobname}.pdb")
  # files.download(f"{pdb_id}_{chain_id}")





FileNotFoundError: ignored